In [5]:
%load_ext autoreload
%autoreload 2


import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline

mp.jupyter.init()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
print(pkg.markdown)

# Current Population Survey, 2023
`census.gov-cps-2023-basic_monthly-1.1.3` Last Update: 2023-03-06T16:32:21

_The Current Population Survey (CPS) is the primary source of labor force statistics for the population of the United States._


## Documentation Links

* [Subject Definitions](https://www.census.gov/programs-surveys/cps/technical-documentation/subject-definitions.html) Descriptions of major concepts.
* [User Notes](https://www.census.gov/programs-surveys/cps/technical-documentation/user-notes.html) Various technical documentation.
* [Homepage](https://www.census.gov/programs-surveys/cps.html) 
* [Data Dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2023/basic/2023_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt) Original text of the data dictionary

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[_labels](data/labels.csv)**. Text descriptions of the intege

In [3]:
pkg

# Current Population Survey, 2023
`census.gov-cps-2023-basic_monthly-1.1.3` Last Update: 2023-03-06T16:32:21

_The Current Population Survey (CPS) is the primary source of labor force statistics for the population of the United States._


## Documentation Links

* [Subject Definitions](https://www.census.gov/programs-surveys/cps/technical-documentation/subject-definitions.html) Descriptions of major concepts.
* [User Notes](https://www.census.gov/programs-surveys/cps/technical-documentation/user-notes.html) Various technical documentation.
* [Homepage](https://www.census.gov/programs-surveys/cps.html) 
* [Data Dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2023/basic/2023_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt) Original text of the data dictionary

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[labels](data/labels.csv)**. Text descriptions of the integer values of categorical variables.
* **[cps202301](notebooks/Extract.ipynb#df)**. Basic Monthly Current Population Survey, January 2023

## References
<ul><li> <strong><a href="https://www2.census.gov/programs-surveys/cps/datasets/2023/basic/jan23pub.csv">cps202301_source</a></strong>. </li><li> <strong>variables</string>, <em>data/variables.csv</em>. All of the variables and their cleaned text descriptions</li><ul>

In [8]:
dd = pkg.reference('data_dictionary_text')
dd.resolved_url.get_resource().fspath

AttributeError: 'NoneType' object has no attribute 'resolved_url'

In [5]:
dd = pkg.reference('data_dictionary_text')
with dd.resolved_url.get_resource().fspath.open(encoding='latin1') as f:
    lines = f.readlines()
 
end_line = next(i for i, l in enumerate(lines) if l.startswith('End of') )

# Split out industry codes
dd_lines = lines[:end_line]
indst_code_lines = lines[end_line: ]


In [15]:
# Classify lines in the data dictionary 

import re

def tok(l):
    l = re.sub(r'\t+', lambda m: f" tabs{len(m[0])} ", l)
    l = re.sub(r'\s\s\s+', lambda m: f" spaces{len(m[0])} ", l)
    return [e for e in re.split(r'\b', l.rstrip() ) if e not in  (' ','', '\n')]



def is_int(t):
    try:
        return str(int(t)) == str(t)
    except ValueError:
        return False

def shape(toks):
    s = []
    for t in toks:
        if t.startswith('tabs'):
            s.append('T')
        elif all([e==' ' for e in t]) or t.startswith('spaces'):
            s.append('S')
        elif all([str.isdigit(e) for e in t]):
            s.append('9')
        elif t.strip() == '-':
            s.append('-')
        elif not all([str.isalnum(e) for e in t]):
            s.append('.')
        elif  all([str.isalnum(e) for e in t]):
            s.append('A')
        else:
            s.append('?')
            
    return ''.join(s)
     
def is_variable_intro(s):
    return (s.startswith('AT9') or s.startswith('ATS9')) and (s.endswith('9-9') or s.endswith('9.9') )
    
def is_categorical_value(s, var_name):
    """
    'T9T9.9A' is for money lines ie, '5	12,500 TO 14,999'
    'T9.A' is for an error in the formating, ie, '1 = Asian Indian' for PRDASIAN
    'T9AA' is for GEDIV, ie '1 NEW ENGLAND'
    'T9T9.9' is for GTCBSASZ, ie, 2	100,000 - 249,999
    'T9T9-9' is for PRINUSYR, '02	1950-1959'
    
    
    There are other errors in the file: 
    
        GESTFIPS is in a two column format. 
    """
        
    return re.match(r'[TS]+9[TS]+A', s) \
        or s.startswith('T9T9.9A') \
        or s.startswith('T9.A')  \
        or s.startswith('T9AA') \
        or s.startswith('T9T9.9') \
        or s.startswith('T9T9-9') or s.startswith('T9ST9-9')


def is_universe(l):
    return 'EDITED UNIVERSE' in l

def is_cat_intro(l):
    """The VALID ENTRIES LINE"""
    
    return 'VALID ENTRIES' in l
  
def categorize_line(l, var_name):
    
    if 'FILLER' in l:
        return "FIL"

    t = tok(l)
    s = shape(t)

  
    if not t:
        return "NIL"

    elif is_variable_intro(s):
        return 'VAR'
    elif is_categorical_value(s, var_name):
        return 'CAT'
    elif is_cat_intro(l):
        return 'VE' # "Valid Entries"
    elif is_universe(l):
        return 'EU' # "Edited Universe"
    else:
        return 'UNK'

    
def debug_dump(lines):
    """Print out the data dict file for debugging """
    
    from colorama import Fore, Back, Style
    var_name = None
    
    for l in lines:

        c = categorize_line(l, None)
        
        if c == 'VAR':
            toks = tok(l)
            try:
                var_name, size, *text, start, _, end = filter_line(toks)
                var_desc = ' '.join(text)
            except ValueError as e:
                print("ERROR",e,l)
        
        if c == 'NIL':
            continue
        elif c == 'UNK':
            pre = Fore.RED + shape(tok(l)) + Style.RESET_ALL
        else:
            pre = str.ljust(c, 8)
        
        
        
        print(pre, l.rstrip())
        
def filter_line(toks):
    """Remove tab and space tokens"""
    
    def remove_f(v):
        if v.startswith('tabs') or v.startswith('spaces'):
            return True
        
        if not v.strip():
            return True
        
        return False
    
    return [ t for t in toks if not remove_f(t) ]
        
def process(lines):  
    
    from collections import defaultdict
    
    vard = {} # Var descriptions
    vvd = defaultdict(dict) # var-val descriptions
    
    in_var = False # Send var name line, but note var values
    in_cat = False # Seen first var value, but not next var name
    
    var_name  = None
    var_val = None
    
    for i, l in enumerate(lines):
        c = categorize_line(l, var_name)
        toks = tok(l)

        if c == 'VAR':
            in_var = True 
            in_cat = False
            try:
                var_name, size, *text, start, _, end = filter_line(toks)
                var_desc = ' '.join(text)
                vard[var_name] = var_desc
            except ValueError:
                print(i, l)
                print(toks)
                print(filter_line(toks))
                raise
        elif c in ('VE','EU'):
            in_var = False 
            in_cat = False
    
        elif c == 'CAT':
            in_var = False 
            in_cat = True
            
            var_val, *text = filter_line(toks)
            
            val_desc = ' '.join(text)
            
            try:
                vvd[var_name][int(var_val)] = val_desc.strip('= ')
                
            except ValueError:
                print("ERROR, CAT", c, var_name, "|", var_val.strip(), "|", val_desc.strip())
            
        elif var_name is not None: # continuation lines
            text = ' '.join(filter_line(toks))
            
            if in_var:
                vard[var_name] = (vard[var_name] +" "+text).strip()
            elif in_cat:
                vvd[var_name][int(var_val)] = (vvd[var_name][int(var_val)]+" "+text).strip()
                    
            elif len(toks):
                pass
                # These are notes, but also sometime text that ought to be attached to the
                # record
                #print("ERROR, Cont Line", i,  toks)
        
            
    return vvd, vard
  
vvd, vard = process(dd_lines)

import pandas as pd
var_vals = pd.DataFrame([ dict(column=var_name.lower(), label=desc, code=var_val) for var_name, v in vvd.items() for var_val, desc in v.items()])


var_vals.to_csv('../data/labels.csv', index=False)
var_vals.sample(10).head()

,column,label,code
5,hurespli,MAX VALUE,99
121,hrhtype,"PRIMARY FAMILY HHLDER - RP IN AF , UNMAR .",5
1366,peerncov,NO,2
1437,pehgcomp,9th grade,5
106,hutypc,UNUSED LINE OF LISTING SHEET,7


In [12]:
# Use OpenAi to re-write the variable descriptions
prompt = """
You will clean up text by performing the following operations

- Organize the text into full English sentences, with propery capitalization and punctuation
- Fix gramatical errors.
- Turn sentence fragments into full sentences
- Remove any words or other characters that can be included in a sentence. 

Here the lines of text to clean up. The first word of each line, before the ":" is the variable name, and the text after the ":" is the text you should clean up. In your output, 
print each variable name, then ":", then the cleaned up text. 

{text_lines} """

from tqdm.auto import tqdm

def openai_one_completion(prompt):
    """Call the OpenAI completions interface to re-write the extra path for a census variable
    into an English statement that can be used to describe the variable."""

    import os
    import openai

    openai.api_key = os.getenv("OPENAI_API_KEY")

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.7,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0.2,
    )

    return response["choices"][0]["text"].strip()

def update_vard():
    """Call Open AI to clean the descriptions"""
    
    
    text_lines = ''

    from more_itertools import chunked

    chunks = list(chunked(list(vard.items()), n=20))

    resp = []

    for chunk in tqdm(chunks):
        text_lines = '\n'.join([ k+":"+v for k, v in chunk])
        p = prompt.format(text_lines=text_lines)

        resp.append(openai_one_completion(p))

    upd_vard = {}

    from itertools import chain
    for g in tqdm(list(chain(resp))):
        for line in g.splitlines():
            try:
                k, v = line.split(':',1)
                upd_vard[k] = v
            except ValueError as e:
                pass
            
    return upd_vard

from pathlib import Path

fn = Path('../data/variables.csv')

if not fn.exists():

    if not "upd_vard" in list(locals()):
        pass upd_vard = update_vard()
        
    rows = [ dict(column=k.lower(), desc=v) for k,v in upd_vard.items()]
    dd = pd.DataFrame(rows)
    dd['desc'] = dd.desc.str.strip()
    
    dd.to_csv(fn index=False)

else:
    print(fn, "exists, skipping")

../data/variable_descriptions.csv exists, skipping


In [7]:
varis = pkg.reference('variables').dataframe()
labels = pkg.resource('_labels').dataframe()

In [15]:
labels = labels[~labels.label.isin(['MIN VALUE','MAX VALUE'])]
labels.to_csv('../data/labels.csv', index=False)
labels

,column,label,code
6,hufinal,FULLY COMPLETE CATI INTERVIEW,1
7,hufinal,PARTIALLY COMPLETED CATI INTERVIEW,2
8,hufinal,COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH,3
9,hufinal,"PARTIAL , NOT COMPLETE AT CLOSEOUT",4
10,hufinal,"LABOR FORCE COMPLETE , SUPPLEMENT INCOMPLETE ...",5
...,...,...,...
1564,prdasian,Korean,5
1565,prdasian,Vietnamese,6
1566,prdasian,Other,7
1567,ptnmemp2,Yes,1
